In [ ]:
import pandas as pd

csv_file = 'torgo.csv'

# Read CSV file into pandas DataFrame
df = pd.read_csv(csv_file)

# Display the DataFrame (optional)
print(df)

In [16]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import pandas as pd
from tqdm import tqdm
import librosa

# Load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

# Load the CSV file into a DataFrame
df = pd.read_csv("torgo.csv")

# List of speakers to process
speakers_to_process = [
    'F01',  # F01 training set is wrong
    'F03', 
    'F04', 
    'M01', 
    'M02', 
    'M03', 
    'M04', 
    'M05'
]

# Function to process each row
def process_row(row):
    if row['speaker_id'] in speakers_to_process:
        audio_path = row['audio']
        
        # Load audio and tokenize
        audio_array, sampling_rate = librosa.load(audio_path, sr=None)
        input_values = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_values
        
        # Retrieve logits
        with torch.no_grad():
            logits = model(input_values).logits
        
        # Decode
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        return transcription
    else:
        return ""

# Apply processing function to each row with tqdm
tqdm.pandas()
df['phoneme'] = df.progress_apply(process_row, axis=1)

# Print inference results
for idx, row in tqdm(df.iterrows(), total=len(df)):
    print(f"Audio: {row['audio']}")
    print(f"Transcription: {row['phoneme']}")
    print()
    
# Save the DataFrame with phoneme transcriptions to a CSV file
df.to_csv('torgo_phoneme.csv', index=False)

/home/jinda/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 15%|█▌        | 2480/16394 [00:00<00:01, 12715.26it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0006.wav
Transcription: d e r t

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0008.wav
Transcription: a s ɛ ɪ n n ə i l i uː r ɑ s uː ɑ a h i v ɛ n k

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0009.wav
Transcription: p aː

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0010.wav
Transcription: aː b

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0011.wav
Transcription: m eɪ

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0012.wav
Transcription: i

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0013.wav
Transcription: n o

Audio: /home/data1/capstone/content/downloads/Torgo//F01/Session1/wav_arrayMic/0014.wav
Transcription: h iː s l oʊ k eɪ k ɐ s aɪ w ʌ ɪ n ð ɪ oʊ p ə n ɛ ɪ ɡ eɪ

Audio: /home/data1/capstone/content

 31%|███▏      | 5158/16394 [00:00<00:00, 12469.92it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0607.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0608.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0609.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0610.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0611.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0612.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0613.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0614.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//FC02/Session3/wav_arrayMic/0615.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/

 47%|████▋     | 7714/16394 [00:00<00:00, 12649.48it/s]

...

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0121.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0122.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0123.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0124.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0125.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0126.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0127.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0128.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC01/Session1/wav_headMic/0129.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC

 94%|█████████▍| 15432/16394 [00:01<00:00, 12315.82it/s]

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0156.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0157.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0158.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0159.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0160.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0161.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0162.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0163.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC02/Session2/wav_arrayMic/0164.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/

100%|██████████| 16394/16394 [00:01<00:00, 11830.52it/s]


Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0554.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0555.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0556.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0557.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0558.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0559.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0560.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0561.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC04/Session1/wav_headMic/0562.wav
Transcription: 

Audio: /home/data1/capstone/content/downloads/Torgo//MC

In [17]:
!kill -9 772103

: 

: 